## 단어임베딩
http://doc.mindscale.kr/km/unstructured/11.html


- one hot 방식이 아닌 index 방식

In [1]:
import numpy as np

W = np.arange(21).reshape(7,3)
W[2]

C:\Users\Gyu\Anaconda3\envs\py37\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\Gyu\Anaconda3\envs\py37\lib\site-packages\numpy\.libs\libopenblas.PYQHXLVVQ7VESDPUVUADXEVJOBGHJPAY.gfortran-win_amd64.dll
C:\Users\Gyu\Anaconda3\envs\py37\lib\site-packages\numpy\.libs\libopenblas.TXA6YQSD3GCQQC22GEQ54J2UDCXDXHWN.gfortran-win_amd64.dll
  stacklevel=1)


array([6, 7, 8])

In [2]:
idx = np.array([1,0,3,0])
W[idx] # word embedding 방식

array([[ 3,  4,  5],
       [ 0,  1,  2],
       [ 9, 10, 11],
       [ 0,  1,  2]])

In [4]:
class Embedding:
    def __init__(self,W):
        self.params = [W]
        self.grads = [np.zeros_like(W)]
        self.idx = None
        
    def forward(self,idx):
        W, = self.params
        self.idx = idx
        out = W[idx]
        return out
    
    def backward(self,dout): # 먼저 쓰여진 값 덮어쓴다. 중복 문제 발생으로 좋지 않음
        dW, = self.grads
        dW[...] = 0
        dW[self.idx] = dout
        return None
    
    def backward2(self,dout): # 중복 index 있어도 잘 처리, for문 사용으로 느림
        dW, = self.grads
        dW[...] = 0
        for i, word_id in enumerate(self.idx):
            dW[word_id] += dout[i]
        return None
    
    def backward3(self,dout): # 가장 최적화
        dW, = self.grads
        dW[...] = 0
        np.add.at(dW, self.idx, dout)
        return None

#### np.add.at(A,idx,B) 

A배열의 idx 위치에 B배열의 원소 순서대로 더하기, A값 변함

https://rfriend.tistory.com/m/545

In [3]:
A = np.arange(18).reshape(6, 3)
print('A:\n',A)
B = np.arange(12).reshape(4, 3)
print('B:\n',B)
idx = np.array([0, 0, 3, 5])
np.add.at(A, idx, B)

print('result:\n',A)

A:
 [[ 0  1  2]
 [ 3  4  5]
 [ 6  7  8]
 [ 9 10 11]
 [12 13 14]
 [15 16 17]]
B:
 [[ 0  1  2]
 [ 3  4  5]
 [ 6  7  8]
 [ 9 10 11]]
result:
 [[ 3  6  9]
 [ 3  4  5]
 [ 6  7  8]
 [15 17 19]
 [12 13 14]
 [24 26 28]]


In [5]:
W = np.arange(15).reshape(5,3)
print('W:\n',W)

e = Embedding(W)

idx = [0,2,0,4]

# 순전파 호출 : 순전파에서는 index가 중복되어도 문제 없음
out = e.forward(idx)  
print('out:\n',out)

W:
 [[ 0  1  2]
 [ 3  4  5]
 [ 6  7  8]
 [ 9 10 11]
 [12 13 14]]
out:
 [[ 0  1  2]
 [ 6  7  8]
 [ 0  1  2]
 [12 13 14]]


In [6]:
# 역전파 호출 : backward
e.grads = [np.zeros_like(W)]
dout = np.arange(12).reshape(4,3)  # 3 : W.shape[1] 과 같다
print('dout:\n',dout)
print('grads:\n',e.grads)

# idx : [0,2,0,4]
e.backward(dout)   # 중복 인덱스가 있어도 올바르게 처리 , 속도가 빠름
print('grads:\n',e.grads)

dout:
 [[ 0  1  2]
 [ 3  4  5]
 [ 6  7  8]
 [ 9 10 11]]
grads:
 [array([[0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0]])]
grads:
 [array([[ 6,  7,  8],
       [ 0,  0,  0],
       [ 3,  4,  5],
       [ 0,  0,  0],
       [ 9, 10, 11]])]


## RNN(Recurrent Neural Network) : 순환 신경망

#### 순서가 있는 시퀀스 데이터, time series data(시계열 데이터)를 입력하여 예측


In [9]:
import numpy as np

class RNN:
    def __init__(self, Wx,Wh,b):
        self.params = [Wx,Wh,b]
        self.grads = [np.zeros_like(Wx), np.zeros_like(Wh),np.zeros_like(b)]
        self.cache = None
        
    def forward(self, x, h_prev):
        Wx, Wh, b = self.params
        t = np.dot(h_prev, Wh) + np.dot(x,Wx) + b
        h_next = np.tanh(t)
        
        self.cache = (x, h_prev,h_next)
        return h_next
    
    def backward(self, dh_next):
        Wx, Wh, b = self.params
        x, h_prev, h_next = self.cache
        
        dt = dh_next *(1 - h_next**2)  # tanh 미분
        db = np.sum(dt, axis=0) # repeat node 역전파
        
        dWh = np.dot(h_prev.T, dt)
        dh_prev = np.dot(dt,Wh.T)
        
        dWx = np.dot(x.T, dt)
        dx = np.dot(dt, Wx.T)
        
        self.grad[0][...] = dWx
        self.grad[1][...] = dWh
        self.grad[2][...] = db
        
        return dx, dh_prev

## Time RNN 계층 구현 : T개의 RNN 계층으로 구성

<img src="https://cloud.githubusercontent.com/assets/901975/23348727/cc981856-fce7-11e6-83ea-4b187473466b.png" width="700">
<img src="https://cloud.githubusercontent.com/assets/901975/23383681/9943a9fc-fd82-11e6-8121-bd187994e249.png" width="700">


In [ ]:
class TimeRNN:
    def __init__(self, Wx, Wh, b, stateful=False):
        self.params = [Wx, Wh, b]
        self.grads = [np.zeros_like(Wx), np.zeros_like(Wh),np.zeros_like(b)]
        self.layers = None         #   RNN층을 리스트로 저장
        
        self.h, self.dh = None, None 
        # h는 forwward() 호출 시 은닉상태를 저장,
        # dh는 backward() 호출 시 앞 블럭의 은닉 상태의 기울기를 저장
        
        self.stateful = stateful   # RNN 계층 사이에서 은닉 상태를 인계한다
    
    def set_state(self, h):
        self.h = h

    def reset_state(self):
        self.h = None
        
    # 순전파
    def forward(self, xs):
        Wx, Wh, b = self.params
        N, T, D = xs.shape # N : batch size T : sequence length D : input size
        D, H = Wx.shape # D  : input size  H : hidden size
        
        self.layers = []
        hs = np.empty((N,T,D),dtype='f')
        
        if not self.stateful or self.h is None:
            self.h = np.zeros((N,H),dtype('f'))
            
        for t in range(T):
            layer = RNN(*self.params)
            self.h = layer.forward(xs[:,t,:],self.h)
            
            hs[:,t,:] = self.h
            self.layers.append(layer)
            
        return hs
        
    def backward(self,dhs):
        Wx, Wh, b = self.params
        N, T, H = dhs.shape 
        D, H = Wx.shape 
        
        dxs = np.empty((N,T,D),dtype = 'f')
        dh = 0
        grads = [0,0,0]
        
        for t in reversed(range(T)):
            layer = self.layers[t]
            dx,dh = layers.backward(dhs[:,t,:] + dh)  # RNN dx, dh_prev 호출
            dxs[:,t,:] = dx
            
        for i, grad in enumerate(grads):
            self.grads[i][...] = grad
        self.dh = dh
        
        return dxs

In [10]:

def func(Wx, Wh, b):
    print(Wx, Wh, b)
    
params =[10,20,30]    
func(*params)  # 튜플인수 : * 사용

params ={'Wx':100,'Wh':200,'b':300}    
func(**params) # 사전인수 : ** 사용

10 20 30
100 200 300
